In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [11]:
from langchain_mistralai import ChatMistralAI
main_model = ChatMistralAI(model="mistral-small-latest")
mistral_model1 = ChatMistralAI(model="mistral-small-latest")
mistral_model = ChatMistralAI(model="mistral-small-latest")

In [12]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    # model='gpt-5-nano',
    mistral_model1, 
    tools=[square_root]
)

subagent_2 = create_agent(
    # model='gpt-5-nano',
    mistral_model,
    tools=[square]
)

## Calling subagents

In [13]:
from langchain.messages import HumanMessage, ToolMessage


@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    
    return response["messages"][-1].content
@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    #model='gpt-5-nano',
    main_model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [14]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [15]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='7745963d-d7d2-4014-843e-6900a96cea09'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'e1zcAonIu', 'function': {'name': 'call_subagent_1', 'arguments': '{"x": 456}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 175, 'total_tokens': 191, 'completion_tokens': 16}, 'model_name': 'mistral-small-latest', 'model': 'mistral-small-latest', 'finish_reason': 'tool_calls', 'model_provider': 'mistralai'}, id='lc_run--019b9c63-4fd6-73b3-93d8-733ed5def856-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'e1zcAonIu', 'type': 'tool_call'}], usage_metadata={'input_tokens': 175, 'output_tokens': 16, 'total_tokens': 191}),
              ToolMessage(content='The square root of 456.0 is approximately 21.354156504062622.', name='call_subagent_1', id='7e5e07a3-10de-46a3-9300-ab25c7bfab66', tool_call_id='e1zcAonIu'),
  

In [16]:
pprint(response["messages"][-1].content)

('The square root of 456 is approximately 21.354156504062622.\n'
 '\n'
 'Do you need help with anything else?')
